In [12]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, color, img_as_uint

input_dir = 'masks'
output_dir = os.path.join(os.path.dirname(input_dir), 'filtered_masks')
os.makedirs(output_dir, exist_ok=True)

mask_files = glob.glob(os.path.join(input_dir, '*cp_masks.tif'))

for mask_path in mask_files:
    mask = io.imread(mask_path)
    labeled_mask = measure.label(mask)
    props = measure.regionprops(labeled_mask)

    areas = np.array([prop.area for prop in props])
    if len(areas) == 0:
        print(f'{os.path.basename(mask_path)} — No regions found.')
        continue

    mean_area, std_area = np.mean(areas), np.std(areas)
    upper = mean_area - 2 * std_area, mean_area + 2 * std_area
    valid_labels = [prop.label for prop in props if prop.area <= upper]

    filtered_mask = np.isin(labeled_mask, valid_labels) * labeled_mask
    filtered_mask = img_as_uint(filtered_mask)

    # Save to filtered_masks folder with same filename
    filename = os.path.basename(mask_path)
    save_path = os.path.join(output_dir, filename)
    io.imsave(save_path, filtered_mask)
    print(f'Saved filtered mask: {filename} → filtered_masks/')

    # Visualization
    original_rgb = color.label2rgb(labeled_mask, bg_label=0)
    filtered_rgb = color.label2rgb(filtered_mask, bg_label=0)
    filtered_areas = [prop.area for prop in measure.regionprops(filtered_mask)]

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(original_rgb)
    ax[0].set_title('Original Mask')
    ax[1].imshow(filtered_rgb)
    ax[1].set_title('Filtered Mask')
    ax[2].hist(filtered_areas, bins=30, color='teal', edgecolor='black')
    ax[2].set_title('Filtered Area Distribution')
    ax[2].set_xlabel('Area (pixels)')
    ax[2].set_ylabel('Count')

    for a in ax[:2]:
        a.axis('off')

    plt.tight_layout()
    plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()